In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import asyncio
import nats

async def main():
    async def disconnected_cb():
        print('Got disconnected!')

    async def reconnected_cb():
        print(f'Got reconnected to {nc.connected_url.netloc}')

    async def error_cb(e):
        print(f'There was an error: {e}')

    async def closed_cb():
        print('Connection is closed')

    # Connect to NATS with logging callbacks.
    nc = await nats.connect('localhost:4222',
                             error_cb=error_cb,
                             reconnected_cb=reconnected_cb,
                             disconnected_cb=disconnected_cb,
                             closed_cb=closed_cb,
                             )

    async def handler(msg):
        print(f'Received a message on {msg.subject} {msg.reply}: {msg.data}')
        await msg.respond(b'OK')

    sub = await nc.subscribe('help.please', cb=handler)

    resp = await nc.request('help.please', b'help')
    print('Response:', resp)

    await nc.close()

if __name__ == '__main__':
    asyncio.run(main())

Received a message on help.please _INBOX.pHX6wtz1OBLD2Txcn4clcA.pHX6wtz1OBLD2Txcn4clgC8863: b'help'
Response: Msg(_client=<nats client v2.9.0>, subject='_INBOX.pHX6wtz1OBLD2Txcn4clcA.pHX6wtz1OBLD2Txcn4clgC8863', reply='', data=b'OK', headers=None, _metadata=None, _ackd=False, _sid=2)
Got disconnected!
Connection is closed
